In this notebook we will build, test, and deploy a Pages app with a homepage and about page listing all the steps involved in creating a Django app

INITIAL SET UP

• create a directory for our code

• install Django in a new virtual environment

• create a new Django project

• createanewpagesapp

• updateconfig/settings.py

In [ ]:
$ cd ~/Desktop
$ mkdir pages && cd pages
$ pipenv install django~=3.1.0
$ pipenv shell
(pages) $ django-admin startproject config .
(pages) $ python manage.py startapp pages

navigate to the file config/settings.py. Add the pages app at the bottom of the INSTALLED_APPS setting:

In [ ]:
# config/settings.py
INSTALLED_APPS = [
    'django.contrib.admin',
    'django.contrib.auth',
    'django.contrib.contenttypes',
    'django.contrib.sessions',
    'django.contrib.messages',
    'django.contrib.staticfiles',
    'pages', # new
]

Start the local web server with runserver.

In [ ]:
(pages) $ python manage.py runserver

And then navigate to http://127.0.0.1:8000/.

# TEMPLATES

Quit the running server with the Control+c command. Then create a directory called templates and an HTML file called home.html.

In [ ]:
(pages) $ mkdir templates
(pages) $ touch templates/home.html

Next we need to update config/settings.py to tell Django the location of our new templates directory. This is a one-line change to the setting 'DIRS' under TEMPLATES.

In [ ]:
# config/settings.py
TEMPLATES = [
    {
        ...
        'DIRS': [str(BASE_DIR.joinpath('templates'))], # new
        ...
}, ]

add a simple headline to our home.html file.

In [ ]:
<!-- templates/home.html -->
<h1>Homepage</h1>

# CLASS BASED VIEWS

Update the pages/views.py file.

In [ ]:
# pages/views.py
from django.views.generic import TemplateView
class HomePageView(TemplateView): template_name = 'home.html'

we’ve capitalized our view, HomePageView, since it’s now a Python class. Classes, unlike functions, should always be capitalized47 . The TemplateView already contains all the logic needed to display our template, we just need to specify the template’s name.

# URLS

The last step is to update our URLConfs.  We need to make updates in two locations. First, we update the config/urls.py file to point at our pages app and then within pages we match views to URL routes.
Let’s start with the config/urls.py file.

In [ ]:
# config/urls.py
from django.contrib import admin
from django.urls import path, include # new
urlpatterns = [
    path('admin/', admin.site.urls),
    path('', include('pages.urls')), # new
]

We add include on the second line to point the existing URL to the pages app. Next create an app-level urls.py file.

In [ ]:
(pages) $ touch pages/urls.py

In [ ]:
# pages/urls.py
from django.urls import path from .views import HomePageView
urlpatterns = [
    path('', HomePageView.as_view(), name='home'),
]

And we’re done! Start up the local web server with python manage.py runserver and navigate to http://127.0.0.1:8000/ to see our new homepage.

# About Page

Quit the server with Control+c and create a new template called about.html.

In [ ]:
(pages) $ touch templates/about.html

In [ ]:
<!-- templates/about.html -->
<h1>About page</h1>

Create a new view for the page

In [ ]:
# pages/views.py
from django.views.generic import TemplateView
class HomePageView(TemplateView):
   template_name = 'home.html'

class AboutPageView(TemplateView): # new
   template_name = 'about.html'

And then import the view name and connect it to a URL at about/.

In [ ]:
# pages/urls.py
from django.urls import path
from .views import HomePageView, AboutPageView # new
urlpatterns = [
    path('about/', AboutPageView.as_view(), name='about'), # new
    path('', HomePageView.as_view(), name='home'),
]

Start up the web server with python manage.py runserver. Navigate to http://127.0.0.1:8000/about and the new About page is visible.

# Extending Templates

Let’s create a base.html file containing a header with links to our two pages.

In [ ]:
(pages) $ touch templates/base.html

In [ ]:
<!-- templates/base.html -->
<header>
<a href="{% url 'home' %}">Home</a> | <a href="{% url 'about' %}">About</a>
</header>
{% block content %}
{% endblock content %}

Let’s update our home.html and about.html files to extend the base.html template. That means we can reuse the same code from one template in another template.

In [ ]:
<!-- templates/home.html -->
{% extends 'base.html' %}
{% block content %}
<h1>Homepage</h1>
{% endblock content %}

In [ ]:
<!-- templates/about.html -->
{% extends 'base.html' %}
{% block content %}
<h1>About page</h1>
 {% endblock content %}

start up the server with python manage.py runserver and open up our webpages again at http://127.0.0.1:8000/ and http://127.0.0.1:8000/about you’ll see the header is magically included in both locations.

# Tests

If you look within our pages app, Django already provided a tests.py file we can use. Open it
and add the following code:

In [ ]:
# pages/tests.py
from django.test import SimpleTestCase
class SimpleTests(SimpleTestCase):
      def test_home_page_status_code(self):
           response = self.client.get('/')
           self.assertEqual(response.status_code, 200)

      def test_about_page_status_code(self):
          response = self.client.get('/about/')
          self.assertEqual(response.status_code, 200)

We’re using SimpleTestCase53 here since we aren’t using a database. If we were using a database, we’d instead use TestCase54. Then we perform a check if the status code for each page is 200, which is the standard response for a successful HTTP request55.

To run the tests quit the server Control+c and type python manage.py test on the command line.

In [ ]:
(pages) $ python manage.py test
System check identified no issues (0 silenced).
..
----------------------------------------------------------------------
Ran 2 tests in 0.014s
OK

# Git and GitHub

In [ ]:
(pages) $ git init
(pages) $ git status
(pages) $ git add -A
(pages) $ git commit -m "initial commit"

Over on GitHub create a new repo56 called pages-app and make sure to select the “Private” radio button. Then click on the “Create repository” button.
On the next page, scroll down to where it says “...or push an existing repository from the command line.” Copy and paste the two commands there into your terminal.
It should look like the below albeit instead of wsvincent as the username it will be your GitHub username.

In [ ]:
(pages) $ git remote add origin https://github.com/wsvincent/pages-app.git
(pages) $ git push -u origin master

# PRODUCTION

You can sign up for a free Heroku60 account on their website. After you confirm your email Heroku will redirect you to the dashboard section of the site.
Now we need to install Heroku’s Command Line Interface (CLI) so we can deploy from the command line. Currently, we are operating within a virtual environment for our Pages project but we want Heroku available globally, that is everywhere on our machine. An easy way to do so is open up a new command line tab–Command+t on a Mac, Control+t on Windows–which is not operating in a virtual environment. Anything installed here will be global.

On Windows, see the Heroku CLI page61 to correctly install either the 32-bit or 64-bit version. If you are using Linux there are specific install instructions62 available on the Heroku website.

Once installation is complete you can close our new command line tab and return to the initial
tab with the pages virtual environment active.
Type the command heroku login and use the email and password for Heroku you just set.

In [ ]:
(pages) $ heroku login
Enter your Heroku credentials:
Email: will@learndjango.com
Password: *********************************
Logged in as will@learndjango.com

Here is the deployment checklist:
• installGunicorn

• addaProcfilefile

• updateALLOWED_HOSTS

Gunicorn can be installed using Pipenv.

In [ ]:
(pages) $ pipenv install gunicorn==19.9.0

Create the Procfile now.

In [ ]:
(pages) $ touch Procfile

Open the Procfile with your text editor and add the following line.

In [ ]:
web: gunicorn config.wsgi --log-file -

The ALLOWED_HOSTS63 setting represents which host/domain names our Django site can serve. This is a security measure to prevent HTTP Host header attacks, which are possible even under many seemingly-safe web server configurations. For now, we’ll use the wildcard asterisk, *, which means all domains are acceptable. Later in the book we’ll see how to explicitly list the domains that should be allowed.
Within the config/settings.py, scroll down to ALLOWED_HOSTS and add a '*' so it looks as follows:

In [ ]:
# config/settings.py
ALLOWED_HOSTS = ['*']

Use git status to check our changes, add the new files, and then commit them:

In [ ]:
(pages) $ git status
(pages) $ git add -A
(pages) $ git commit -m "New updates for Heroku deployment"
(pages) $ git push -u origin master

# DEPLOYMENT

Our process will be as follows:
• create a new app on Heroku

• disable the collection of static files (we’ll cover this in a later chapter)

• push the code up to Heroku

• start the Heroku server so the app is live

• visit the app on Heroku’s provided URL

In [ ]:
(pages) $ heroku create
Creating app... done, ￿ fathomless-hamlet-26076
https://fathomless-hamlet-26076.herokuapp.com/ |
https://git.heroku.com/fathomless-hamlet-26076.git

We only need to do one set of Heroku configurations at this point, which is to tell Heroku to ignore static files like CSS and JavaScript which Django by default tries to optimize for us.

In [ ]:
(pages) $ heroku config:set DISABLE_COLLECTSTATIC=1

Now we can push our code to Heroku.

In [ ]:
(pages) $ git push heroku master

Finally, we need to make our Heroku app live. As websites grow in traffic they need additional Heroku services but for our basic example we can use the lowest level, web=1, which also happens to be free! Type the following command:

In [ ]:
(pages) $ heroku ps:scale web=1

We’re done! The last step is to confirm our app is live and online. If you use the command heroku open your web browser will open a new tab with the URL of your app:

In [ ]:
(pages) $ heroku open

You do not have to log out or exit from your Heroku app. It will continue running at this free tier on its own, though you should type exit to leave the local virtual environment